In [1]:
from collections import Counter

import pandas as pd

from sklearn.metrics import cohen_kappa_score

In [2]:
# Loading the main dataset with mine scores
df_main = pd.read_csv('data/dataset_classification.csv', index_col=0)

# printing some information about the main dataset
print("Number of rows:", len(df_main))
print("Number of sentences:", len(set(df_main['question_id'].to_list())))
print()

# Loading the interraters dataset with their scores
df_interraters = pd.read_csv("data/dataset_interraters.csv", index_col=0)

# printing some information about the interrater dataset
interraters = list(set(df_interraters['respondent_id'].to_list()))
print("Number of interraters:", len(interraters))
print("Countrate per interrater:", len(df_interraters)/len(interraters))
print("Number of questions done:", len(df_interraters))

Number of rows: 213
Number of sentences: 213

Number of interraters: 5
Countrate per interrater: 17.2
Number of questions done: 86


In [3]:
rater1 = []
rater2 = []

for i, (index, row) in enumerate(df_interraters.iterrows()):

    other_ranking = df_main[df_main['question_id'] == row['question_id']].to_dict()
    key = list(other_ranking['respondent_id'].keys())[0]

    rater1.append(other_ranking['ranking'][key].split(',')[0])
    rater2.append(row['ranking'].split(',')[0])
    if rater1[-1] != rater2[-1]:
        if (row['opmerkingen'] == "Ik vind dat ze alle 3 even goed zijn" and other_ranking['opmerkingen'][key] == "Ik vind dat ze alle 3 even goed zijn"):
            rater1[-1] = rater2[-1]
        elif (row['opmerkingen'] == "Ik twijfel tussen nummer 1 en 2" and other_ranking['opmerkingen'][key] == "Ik twijfel tussen nummer 1 en 2") and \
            sorted(row['ranking'].split(',')[0:2]) == sorted(other_ranking['ranking'][key].split(',')[0:2]):
            rater2[-1] = rater1[-1]
        elif (row['opmerkingen'] == "Ik twijfel tussen nummer 1 en 2" or other_ranking['opmerkingen'][key] == "Ik twijfel tussen nummer 1 en 2"):
            if other_ranking['opmerkingen'][key] == "Ik twijfel tussen nummer 1 en 2" and row['ranking'].split(',')[0] in other_ranking['ranking'][key].split(',')[0:2]:
                rater1[-1] = rater2[-1]
            elif row['opmerkingen'] == "Ik twijfel tussen nummer 1 en 2" and other_ranking['ranking'][key].split(',')[0] in row['ranking'].split(',')[0:2]:
                rater2[-1] = rater1[-1]
print(cohen_kappa_score(rater1, rater2))

0.6315163056415138


In [4]:
# Opmerkingen interraters
Counter(df_interraters['opmerkingen']).most_common()

[('Geen extra informatie', 61),
 ('Ik twijfel tussen nummer 2 en 3', 12),
 ('Ik twijfel tussen nummer 1 en 2', 8),
 ('Ik vind dat ze alle 3 even goed zijn', 5)]

In [5]:
# Opmerkingen main
Counter(df_main['opmerkingen']).most_common()

[('Geen extra informatie', 113),
 ('Ik twijfel tussen nummer 1 en 2', 39),
 ('Ik twijfel tussen nummer 2 en 3', 33),
 ('Ik vind dat ze alle 3 even goed zijn', 28)]